# Plastic Recycle Classification

Notebook ini berfungsi sebagai resource untuk pembuatan model klasifikasi gambar plastik berdasarkan kode recycle-nya.
Notebook ini mencakup:
1. Pengaturan struktur folder dan pemetaan gambar berdasarkan tipe objek.
2. Penggunaan `ImageDataGenerator` dari Keras untuk mempersiapkan dataset
3. Melatih model image classification menggunakan MobileNetV2.
4. Evaluasi model menggunakan dataset test.
   
Dengan notebook ini, \ dapat mempermudah proses pembuatan model asli untuk klasifikasi plastik berdasarkan kode recycle.


## Import Library

In [ ]:
import os
import shutil
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

## Mengatur Struktur Folder dan Memetakan Gambar (incase butuh code buat metain folder)

In [ ]:
# Path ke folder gambar asli
source_dir = 'path_to_your_images'
# Path ke folder target
target_dir = 'path_to_target_dataset'

# Dictionary untuk mapping tipe objek ke kategori recycle
object_to_recycle = {
    'Botol Minyak': 'PET',
    'Botol Plastic': 'PET',
    'tempat makan ovenproof': 'PET',
    'Botol Susu (kecil)': 'HDPE',
    'Botol Jus': 'HDPE',
    'Kemasan Mentega': 'HDPE',
    'Botol Detergen': 'PVC',
    'Shampoo': 'PVC',
    'Pipa': 'PVC',
    'Mika': 'PVC',
    'Kresek': 'LDPE',
    'botol yang ditekan': 'LDPE',
    'pembukus biscuit': 'PP',
    'Botol Obat': 'PP',
    'Sedotan': 'PP',
    'Botol Minuman': 'PP',
    'Susu Bayi': 'PP',
    'CD': 'PS',
    'Styrofoam': 'PS',
    'wadah makanan beku': 'PS'
}

# Pastikan folder target ada
os.makedirs(target_dir, exist_ok=True)

# Function untuk memindahkan gambar
def move_images(source_dir, target_dir, object_to_recycle):
    for file_name in os.listdir(source_dir):
        for object_type, recycle_category in object_to_recycle.items():
            # Periksa apakah tipe objek ada dalam nama file
            if object_type.lower().replace(' ', '_') in file_name.lower():
                # Path target untuk kategori recycle
                target_sub_dir = os.path.join(target_dir, recycle_category, object_type)
                os.makedirs(target_sub_dir, exist_ok=True)
                # Pindahkan file ke folder yang sesuai
                shutil.move(os.path.join(source_dir, file_name), os.path.join(target_sub_dir, file_name))
                break

# Pindahkan gambar berdasarkan mapping
move_images(source_dir, target_dir, object_to_recycle)

print("Images moved successfully!")


## Menyiapkan Data Generator

Menggunakan ImageDataGenerator untuk Memuat Dataset

In [ ]:
train_path = ''
val_path = ''
# ImageDataGenerator for training with data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,  # Normalize pixel values to [0, 1]
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2  # 20% of data used for validation
)

# Training data generator
train_generator = train_datagen.flow_from_directory(
    target_dir,
    target_size=(224, 224),  # Resize images to 224x224
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

# Validation data generator
validation_generator = train_datagen.flow_from_directory(
    target_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)



## Melatih Model Dengan MobileNetV2

In [ ]:
# Load the pretrained MobileNetV2 model
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add new layers on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(train_generator.num_classes, activation='softmax')(x)

# Define the new model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the layers of the base model
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator
)


## Evaluasi Model

In [ ]:
# Test data generator
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    'path_to_test_dataset',  # Adjust this to your test dataset path
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

# Evaluate the model
loss, accuracy = model.evaluate(test_generator)
print(f'Test Accuracy: {accuracy*100:.2f}%')